<a href="https://colab.research.google.com/github/TWT-25/LLM/blob/main/Dialogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install torch torchvision torchaudio
!pip install transformers[torch] accelerate -U

from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device found")

import warnings
warnings.filterwarnings('ignore')

import pandas as pd



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 122.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting unin

In [ ]:
# Load DialoGPT tokenizer and model
model_name = "microsoft/DialoGPT-medium"
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side = "right")
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

In [ ]:
tokenizer

GPT2TokenizerFast(name_or_path='microsoft/DialoGPT-medium', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [ ]:
# Load the dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")
df = dataset['train'].to_pandas()

print(df.head())

README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

                                             Context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            Response  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  


In [ ]:
df['input'] = df['Context'] + " " + df['Response']
df.head()


,Context,Response,input
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",I'm going through some things with my feelings...
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",I'm going through some things with my feelings...
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,I'm going through some things with my feelings...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...,I'm going through some things with my feelings...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...,I'm going through some things with my feelings...


In [ ]:
df.shape, df.memory_usage()

((3512, 3),
 Index         128
 Context     28096
 Response    28096
 input       28096
 dtype: int64)

In [ ]:
conv = Dataset.from_pandas(df)
conv

Dataset({
    features: ['Context', 'Response', 'input'],
    num_rows: 3512
})

In [ ]:
conv[0]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

In [ ]:
def chat(inp = None):
  model_cpu = model.to("cpu")
  for step in range (5):
    if not inp:
      new_user_input_ids = tokenizer.encode(input(">> User: ") + tokenizer.eos_token, return_tensors="pt")
    else:
      print(">> User: ", inp[step])
      new_user_input_ids = tokenizer.encode(inp[step]+tokenizer.eos_token, return_tensors="pt")

    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step>0 else new_user_input_ids

    attention_mask = torch.ones(bot_input_ids.shape, dtype=torch.long)

    chat_history_ids = model_cpu.generate(
        bot_input_ids,
        max_length=1000,
        attention_mask=attention_mask,
        pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    response = response.replace("<|endoftext|>", "").strip()  # Manually remove any "<|endoftext|>" remnants

    print(">> DialoGPT: {}".format(response))

In [ ]:
chat()

>> User: Hi im Tina
>> DialoGPT: Hi Tina
>> User: How are you?
>> DialoGPT: I'm good, how are you?
>> User: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
>> DialoGPT: I'm sorry to hear that.
>> User: I've always wanted to fix my issues, but I never get around to it.
>> DialoGPT: I'm sorry to hear that.
>> User: How can I feel better?
>> DialoGPT: I'm sorry to hear that.


In [ ]:
def encode(examples):
  encoded = tokenizer(examples['input'], truncation=True, padding='max_length', max_length=40, return_tensors="pt")
  encoded['labels'] = encoded['input_ids'][:]
  return encoded

In [ ]:
split_data = conv.train_test_split(test_size=0.2)
encoded_data = split_data.map(encode, batched=True)


Map:   0%|          | 0/2809 [00:00<?, ? examples/s]

Map:   0%|          | 0/703 [00:00<?, ? examples/s]

In [ ]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response', 'input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2809
    })
    test: Dataset({
        features: ['Context', 'Response', 'input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 703
    })
})

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="fine_tuned_dislogpt",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    learning_rate=2e-5,
    fp16=True
)


training_args.device


device(type='cuda', index=0)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_data["train"],
    eval_dataset=encoded_data["test"]
)


In [ ]:
pre_eval_results = trainer.evaluate(encoded_data["test"])
pre_val_predictions = trainer.predict(encoded_data["test"].select(range(10)))


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
print(pre_eval_results)

{'eval_loss': 4.967205047607422, 'eval_model_preparation_time': 0.0328, 'eval_runtime': 18.8677, 'eval_samples_per_second': 37.259, 'eval_steps_per_second': 18.656}


In [ ]:
import numpy as np
def generate_response(predictions):
  responses = []
  for idx, pred in enumerate(predictions):
    response = tokenizer.decode(np.argmax(pred, axis=-1), skip_special_tokens=True)
    response
    responses.append(response)

  return responses

In [ ]:
res = generate_response(pre_val_predictions.predictions)
res

[" the not fan bigster.I don not in a relationship program at is me of social and presentations I're seem me feel like out. anxious confident. do I improve out confident? lot",
 " and need afford to get a.We think stand a good with you. her being into a War 3.0 is me like a'm a life to be opinion. is listens cares me have.",
 " the reading, guy, I did the client of you client is? counselor is be different different process. but I'm not sure best of be on.ive me. I'm the",
 ' the and the countrys district. few of years ago. the. a got married, a airport, was a a for aaph.He the city he was been me tests of the happened',
 ' the not fan parent ofI a single, I was ated by my father. boyfriend.I was knew that father.I never to sexual. age. after that school.I that kids,',
 " and I do? I get each other?'m a confused for you is.Iive is to be treated of like a consent.IersionI'm like you to talk a touch with",
 " the is a a hard life time with the paper.I's died up. time he uses a ownieI'saze

In [ ]:
trainer.train()

Step,Training Loss
500,2.964200
1000,2.150200
1500,1.661300
2000,1.360300
2500,1.153400
3000,0.991300
3500,0.889200
4000,0.780400
4500,0.708700
5000,0.673000


TrainOutput(global_step=7030, training_loss=1.1106910678434847, metrics={'train_runtime': 2340.9103, 'train_samples_per_second': 12.0, 'train_steps_per_second': 3.003, 'total_flos': 2038062696038400.0, 'train_loss': 1.1106910678434847, 'epoch': 10.0})

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

34

In [ ]:
post_eval_results = trainer.evaluate(encoded_data["test"])
post_val_predictions = trainer.predict(encoded_data["test"].select(range(10)))

In [ ]:
post_eval_results

{'eval_loss': 1.0713404417037964,
 'eval_model_preparation_time': 0.0328,
 'eval_runtime': 8.4285,
 'eval_samples_per_second': 83.407,
 'eval_steps_per_second': 41.763,
 'epoch': 10.0}

In [ ]:
chat()

>> User: hi im Tina
>> DialoGPT: Hi Tina!
>> User: how are you?
>> DialoGPT: I'm doing well! How are you?
>> User: I am hungry!
>> DialoGPT: me too thanks
>> User: where can i find some food?
>> DialoGPT: I have a couple places I can go to. I live in a small town so I don't have many options. I also don't have many friends. I am very sad that I have so many issues. How are you doing today?
>> User: sorry to hear that! i hope your day gets better
>> DialoGPT: thank you for your question.     I am very glad to hear that you are doing well.
